Pour regarder sim LAM_1500 no_irr vs LMDZ Zoomé Guidé (simus Frédérique 2010-2014)

# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *

In [102]:
# rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load and edit files

LAM output analysis.

simu irr et simu no-irr.

## Area and period selection

In [103]:
# lon_min=-19
# lon_max=12
# lat_min=28
# lat_max=53

date_min = '2010-01-01'
date_max = '2014-12-31'

In [104]:
#filter to keep only certain period
# season='SON'
# sim = sim.where(sim['time.season'] == season, drop=True)
# simirr = simirr.where(simirr['time.season'] == season, drop=True)

## Sims

In [105]:
TS_flag=False

In [106]:
# noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
# irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/irr_2010_2022'
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'

In [ ]:
noirr_dir

sim0 = xr.open_mfdataset(filename)
sim0.attrs['name'] = 'LAM_1500'
sim = sim0.rename({'time_counter':'time'})
sim.attrs["plot_color"] = 'red'
# sim = sim.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))
sim = sim.sel(time=slice(date_min, date_max))


sim['precip'] = sim['precip'] *3600 * 24
sim['precip'].attrs['units'] = 'mm/d'
sim['precip'].attrs['long_name'] = 'Total precipitation'


sim

In [ ]:
filename = '../../../JZ_simu_outputs/ZG_Frederique/*.nc'.format(noirr_dir)


zg = xr.open_mfdataset(filename)
zg.attrs['name'] = 'zg'
zg=zg.rename({'time_counter':'time'})
zg.attrs["plot_color"] = 'blue'

# zg = zg.sel(lon=slice(lon_min, lon_max),lat=slice(lat_max, lat_min))


zg['precip'] = zg['precip'] *3600 * 24
zg['precip'].attrs['units'] = 'mm/d'
zg['precip'].attrs['long_name'] = 'Total precipitation'

zg

In [117]:
#continental fraction mask (LMDZ outputs)
con_mask=sim.mean(dim='time')['contfracOR']>0.95
ip_mask=polygon_to_mask(sim, iberian_peninsula) * con_mask

In [118]:
ip_sim=sim.where(ip_mask)

In [119]:
zg_iLAM=zg.interp_like(sim)
ip_zg=zg_iLAM.where(ip_mask)

# Maps

In [ ]:
#Display map of var for ds
var='precip'
ds=sim
color_map=wet

vmin=0
vmax= 5
# vmin=None
# vmin,vmax=None, None
# cbar_label='Altitude (m)'
cbar_label='Precipitation (mm/d)'
# title='LMDZ ZG simulation'
title='LAM simulation'
poly=None

# map_plotvar(ds[var])
map_ave(ds, var, cmap=color_map, figsize=(8,4), title=title, vmin=vmin, vmax=vmax, clabel=cbar_label, poly=poly)

In [ ]:
#Diff map
var='precip'
ds1=sim
# ds2=ip_era_isim
# ds2=ip_fluxcom_isim
# ds2=gleam_isim
ds2=zg_iLAM

max_value= 2.0
min_value=-2.0
# max_value=None
# min_value=None
# min_value,max_value=None, None
cmap=emb_neutral
clabel='Precipitation difference (mm/d)'
title='Diff : LAM - LMDZ ZG'

map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, cmap=cmap,clabel=clabel, title=title,
             sig=False, sig_viz=3)
# diff=ds1[var]-ds2[var]
# map_seasons(diff, cmap=emb_neutral, vmin=min_value, vmax=max_value, title=title, hex=False

In [ ]:
#map for 4 seasons
var='t2m'
ds1=ip_simirr
ds2=ip_sim
max_value= 0.5
min_value=-0.5
# min_value,max_value=None, None
cmap=emb

diff=ds1[var]-ds2[var]
title='{} diff, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

map_seasons(plotvar, cmap=cmap, vmin=min_value, vmax=max_value, title=title, hex=False)

In [ ]:
# Display 2 maps of var
var = 'SWupSFC'
min_value=None
max_value=None
color_map=wet
ds1=sim_iera
ds2=era
map_two_ds(ds1, ds2, var, vmin=min_value, figsize=(15,6), vmax=max_value, cmap=color_map)

In [ ]:
#Display map of var for ds on restricted area
var='evap'
ds = sim
latmin=36.7
latmax=36.9
lonmin=-6.4
lonmax=-6.2

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=emb_neutral_neutral, vmax=None, title=var)

In [ ]:
#Relative diff map
var='precip'
max_value=None
ds1=cont_fluxcom
ds2=ip_fluxcom
# plt.title('Gross primary production relative difference (%)')
map_rel_diff_ave(ds1, ds2, var, vmax=max_value)

In [ ]:
ds=sim
scale=2000
dist=2
extra_var='norm'
cmap=emb
vmin=None
vmax=1550
vmin,vmax=None, None
poly=ebro
map_moisture_transport(ds, extra_var=extra_var, cmap=cmap, vmin=vmin, vmax=vmax, dist=dist,scale=scale, poly=poly)

# Time series

In [81]:
ip_gpcc_isim.attrs['plot_color']='black'

In [ ]:
var='precip'

ds1=sim
ds2=zg_iLAM

ds_list=[ds1, ds2]


year_min=2010
year_max=2022

ylabel='Precipitation (mm/d)'
# title='Seasonal cycle of {} ({}, {}-{})'.format(var, ds1[var].attrs['units'], year_min, year_max)
title='off'
figsize=(8.5,5)
time_series_ave(ds_list, var, ds_colors=True, year_min=year_min, year_max=year_max, title='{} {}'.format(var, ds1[var].attrs['units']))
seasonal_cycle_ave(ds_list, var, ds_colors=True, year_min=year_min, year_max=year_max, ylabel=ylabel, title=title, figsize=figsize)

# Scatter plots

In [ ]:
var1='precip'
ds1=ip_sim_era_diff_mean
#remove values where precip is smaller than 5
mask=ds1[var1] < 2
ds1=ds1.where(mask)

var2='evap'
# ds2=ip_sim_era_diff
ds2=ds1

scatter_vars(ds1, ds2, var1, var2, reg=False)

In [ ]:
var1='precip'
ds1=ip_sim_gpcc_diff_mean
#remove values where precip is smaller than 2
mask=ds1[var1] < 2
ds1=ds1.where(mask)

var2='evap'
ds2=ip_sim_gleam_diff_mean
ds2=ds2.where(mask)
scatter_vars(ds1, ds2, var1, var2, reg=False)

# Wind

In [ ]:
ds = ip_sim
dist=2
height='10m'
scale=20
vmin=0
vmax=2
map_wind(ds, dist=dist, height=height, vmin=vmin, vmax=vmax, scale=scale)

In [ ]:
ds1 = ip_simirr
# ds1=ds1.sel(time=ds1['time.month'].isin([6,7,8]))
ds2 = ip_sim
# ds2=ds2.sel(time=ds2['time.month'].isin([6,7,8]))
dist=5
height='10m'
scale=0.2
vmin=-0.1
vmax= 0.1
map_wind_diff(ds1, ds2, dist=dist, height=height, vmin=vmin, vmax=vmax, scale=scale)

In [ ]:
# 850hPa wind difference
ds1 = ip_simirr
ds2 = ip_sim
dist=2

#compute wind speed
wind_speed1 = (ds1['u850'][0,:,:]**2 + ds1['v850'][0,:,:]**2)**(1/2)
wind_speed2 = (ds2['u850'][0,:,:]**2 + ds2['v850'][0,:,:]**2)**(1/2)
wind_speed_diff = wind_speed1 - wind_speed2
#extract x and y (u and v) components and reduce dimension
longi=ds['lon'][::dist]
lati=ds['lat'][::dist]
windx1 = ds1['u850'][0,::dist,::dist]
windy1 = ds1['v850'][0,::dist,::dist]
windx2 = ds2['u850'][0,::dist,::dist]
windy2 = ds2['v850'][0,::dist,::dist]
windx_diff = windx1 - windx2
windy_diff = windy1 - windy2


#display map
fig = plt.figure(figsize=(10, 7.5))
ax = plt.axes(projection=ccrs.PlateCarree())
# gridline setup
ax.coastlines()
ax.add_feature(rivers)
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.8)
gl.right_labels = False
gl.top_labels = False
gl.xlocator = plt.MaxNLocator(9)
gl.ylocator = plt.MaxNLocator(9)
wind_speed_diff.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=emb)
ax.quiver(longi, lati, windx_diff, windy_diff, transform=ccrs.PlateCarree(), scale=0.1)
plt.title('850hPa wind speed (m/s) and direction difference')
plt.show()


# Moisture fluxes

In [49]:
ebro_mask=polygon_to_mask(sim, ebro)

In [ ]:
ds=era
mask=ip_mask
mask=polygon_to_mask(ds, iberian_peninsula)
era_budget=moisture_budget(ds, mask)
era_budget

In [ ]:
ds=era_isim
mask=ip_mask
mask=polygon_to_mask(ds, iberian_peninsula)
era_budget=moisture_budget(ds, mask)
era_budget

# Vertical profiles

In [ ]:
filename='../../../JZ_simu_outputs/LAM/HF_outputs_LAM_concat/TS_MO_irr_*.nc'
lvls_irr=xr.open_mfdataset(filename)
lvls_irr.attrs["name"]='irr'
#rename time_counter to time
lvls_irr = lvls_irr.rename({'time_counter':'time'})
lvls_irr

In [ ]:
filename='../../../JZ_simu_outputs/LAM/HF_outputs_LAM_concat/TS_MO_noirr_*.nc'
lvls_noirr=xr.open_mfdataset(filename)
lvls_noirr.attrs["name"]='no_irr'
#rename time_counter to time
lvls_noirr = lvls_noirr.rename({'time_counter':'time'})
lvls_noirr

In [ ]:
filename = '{}/SRF/MO/*history.nc'.format(irr_dir)
orc0 = xr.open_mfdataset(filename)
orc = orc0.rename({'time_counter':'time'})


In [ ]:
orc_interp=orc.interp_like(lvls_irr)
irr_mask = (orc_interp['irrigmap_dyn']/orc_interp['Areas'])>0.1
nonirr_mask = (orc_interp['irrigmap_dyn']/orc_interp['Areas'])<=0.1

In [ ]:
lvls_irr_irrigated=lvls_irr.where(irr_mask)
lvls_irr_irrigated.attrs['name']='irrigated'
lvls_irr_nonirrigated=lvls_noirr.where(nonirr_mask)
lvls_irr_nonirrigated.attrs['name']='non_irrigated'

In [ ]:
ip_mask=polygon_to_mask(lvls_irr, iberic_peninsula)

ip_lvls_irr=lvls_irr.where(ip_mask, drop=False)
ip_lvls_irr.attrs['name']='irr'

ip_lvls_noirr=lvls_noirr.where(ip_mask, drop=False)
ip_lvls_noirr.attrs['name']='no_irr'

ip_lvls_irr_irrigated = lvls_irr.where(irr_mask).where(ip_mask, drop=False)
ip_lvls_irr_irrigated.attrs['name']='irrigated'

ip_lvls_irr_nonirrigated = lvls_irr.where(nonirr_mask).where(ip_mask, drop=False)
ip_lvls_irr_nonirrigated.attrs['name']='nonirrigated'

In [ ]:
# t and q profiles
#select a lon-lat box
latmin=41
latmax=43
lonmin=-2
lonmax=1
var='q'
ds1=simirr.sel(lon=slice(lonmin,lonmax),lat=slice(latmin,latmax))
ds2=sim.sel(lon=slice(lonmin,lonmax),lat=slice(latmin,latmax))

ds_list=[ds1, ds2]

profile_reflevs(ds_list, var)

In [ ]:
# profile
preslevelmax=50
var='ovap'
# select a lon-lat box
latmin=41
latmax=43
lonmin=-2
lonmax=1
# #spain
# latmin=36
# latmax=44
# lonmin=-9.5
# lonmax=3

ds1=ip_lvls_irr_irrigated
ds1= ds1.sel(lon=slice(lonmin,lonmax),lat=slice(latmin,latmax))
#select only JJA season
# ds1=ds1.sel(time=ds1['time.month'].isin([6,7,8]))

ds2=ip_lvls_irr_nonirrigated
ds2=ds2.sel(lon=slice(lonmin,lonmax),lat=slice(latmin,latmax))
#select only JJA season
# ds2=ds2.sel(time=ds2['time.month'].isin([6,7,8]))

ds_list=[ds1, ds2]
profile_preslevs(ds_list, var, preslevelmax=preslevelmax)

In [ ]:
#temp at first level (better than t2m)
months=[6,7,8]
var1=ip_lvls_noirr['ovap'][:,0,:,:].sel(time=simirr['time.month'].isin(months))
var2=ip_lvls_irr['ovap'][:,0,:,:].sel(time=simirr['time.month'].isin(months))
temp_diff=(var2-var1).mean(dim='time')
temp_diff.attrs['units']='K'
# p_values = xr.apply_ufunc(
#     lambda x, y: ttest_ind(x, y, axis=0, nan_policy='omit').pvalue, 
#     var1, var2,
#     input_core_dims=[['time'], ['time']],
#     output_core_dims=[[]],
#     vectorize=True,
#     dask='parallelized',
#     output_dtypes=[float]
# )
# mask=p_values<0.05
# temp_diff=temp_diff.where(mask)
map_plotvar(temp_diff, in_cmap=emb, in_vmax=None, title='First level temperature (K) diff (irr-noirr)')

# Testing seasonnal dataset

In [ ]:
ds=ip_simirr
var='evap'
ds_seasons=seasonnal_ds_list(ds)
# time_series_ave(ds_seasons, var)
for ds_season in ds_seasons:
    map_ave(ds_season, var, cmap=wet, vmin=0, vmax=5, title='{} ({})'.format(var, ds_season.attrs['name']))